Trabajo práctico final: detección de cáncer de mamas con machine learning
===================================================

En este trabajo vamos a utilizar técnicas de machine learning con el objetivo de detectar casos de cáncer de mamas, a partir de datos reales medidos en células extraídas como muestras. Estos datos incluyen aspectos como diámetro de la célula, rugosidad, simetría, etc, medidos en las 3 dimensiones.

El problema:
============

Se poseen mediciones de distintos aspectos en células de tumores tomadas como muestras para casos donde se sospecha de la presencia de cáncer de mamas. Se desea aprender una función que permita diagnosticar la presencia o ausencia de cáncer de mamas (es decir, determinar si el tumor es maligno o benigno) a partir de dicha información.

La solución:
============

Como las entradas tienen 30 dimensiones, no es posible armar un gráfico que muestre los puntos con todos sus datos, y allí ver su agrupación. No vamos a poder visualmente darnos cuenta si son realmente clasificables.

Para aprender la función deseada vamos a intentar utilizar el algoritmo K-Vecinos para clasificación, implementado en la biblioteca Scikit-Learn. Como el algoritmo requiere que se defina el parámetro K (la cantidad de vecinos), vamos a evaluar el algoritmo con distintos valores de K y elegir el valor que mejores resultados obtenga.

Como la muestra de casos no está balanceada (hay un porcentaje mayor de casos benignos), para evaluar los resultados vamos a utilizar 3 medidas: precisión (porcentaje de muestras bien clasificadas), accuracy y recall (medidas explicadas en clase, que aseguran que el algoritmo no nos haga "trampa" prediciendo siempre las clases más probables).

Requerimientos:
===============

Para poder resolver esta entrega van a requerir las siguientes herramientas instaladas:

**Ipython Notebook**

La herramienta utilizada para editar y ejecutar este documento.

Instalación en linux: 
    
    sudo pip install "ipython[notebook]"

Para windows, recomendamos instalarlo utilizando Conda.

**Numpy**

Biblioteca de python para cálculo numérico rápido (está programada en C en su mayor parte), que vamos a utilizar para armar los arrays de datos de entrada y salida.

Instalación en linux:

    sudo apt-get install python-numpy

Para windows, recomendamos instalarlo utilizando Conda.

**Bokeh**

Biblioteca de python para visualizaciones interactivas, que vamos a utilizar para visualizar gráficas.

Instalación en linux: 
(incluye instalar headers y compiladores necesarios para dependencias de bokeh)

    sudo apt-get install python-dev build-essential
    sudo pip install bokeh

Para windows, recomendamos instalarlo utilizando Conda.

**SciKit-Learn**

Biblioteca de python para machine learning, que utilizaremos para el algoritmo de clasificación.

Instalación en linux: 

    sudo apt-get install python-sklearn

Para windows, recomendamos instalarlo utilizando Conda.

Ejecución y entrega:
====================

Para abrir y ejecutar este notebook, deben bajar los archivos ```tpfinal.ipynb``` y ```wdbc.data``` del repositorio, y luego desde la consola, posicionarse en la carpeta donde se encuentren los archivos y ejecutar:

    ipython notebook
    
Esto abrirá una nueva pestaña en el navegador, en la que van a ver listados los notebooks presentes. Haciendo click en ```tpfinal.ipynb``` se abre este mismo notebook, pero ya con capacidad para editarlo y ejecutar las celdas de código (las celdas se ejecutan eligiendo la celda y presionando ```shift+enter```. Las celdas se editan haciéndoles doble click).

Sobre este documento deben resolver los ejercicios que se encuentran detallados (incluyen agregar código para resolver secciones, gráficos, y obtención de métricas), guardando las modificaciones. Cuando lo consideren terminado, deben subir el ```tpfinal.ipynb``` modificado a sus propios repositorios.

In [ ]:
import csv

import numpy as np

from bokeh.plotting import output_notebook, show, figure

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler

output_notebook()

Código de ayuda y ejercicios:
=============================


Hay dos clases posibles de tumores: benignos y malignos. Definimos colores para cada clase, que se usan en las gráficas.

In [ ]:
colores_clases = 'green', 'red'

Lo primero que tenemos que hacer es leer los datos que vamos a usar para entrenar y testear al clasificador, que están en un archivo con formato csv. Los leemos y convertimos a arrays de numpy, que son mucho más eficientes y rápidos que usar listas de python normales.

El archivo de datos tiene muchas columnas. Las entradas van desde la columna 2 en adelante. La salida es la columna 1. Y la columna 0 es un id de referencia del caso, que no nos interesa.

Las entradas son todos números, y la salida son las letras "B" y "M". Deseamos convertir eso a números, que se correspondan con 0 y 1.

In [ ]:
with open('wdbc.data') as archivo:
    datos_leidos = list(csv.reader(archivo))

entradas = np.array([map(float, fila[2:]) for fila in datos_leidos])
salidas = np.array([0 if fila[1] == "B" else 1 for fila in datos_leidos])


Inspeccionamos los datos que obtuvimos:

Tenemos 569 casos. Las entradas tienen 30 features (columnas). Y las salidas son simplemente un vector.

In [ ]:
print entradas.shape 
print salidas.shape

Como no es posible graficar las muestras debido a que tienen más de 3 dimensiones, si queremos graficarlas para analizar de forma naive qué tan separables son, podemos hacerlo tomando de a pares de dimensiones.

El siguiente gráfico hace eso eligiendo dos features al azar, y como podemos ver, si tomamos solo esas dimensiones los datos no son tan separables por sus clases.

In [ ]:
feature1 = 19
feature2 = 29

salidas_a_colores = [colores_clases[v] for v in salidas]

figura = figure()

figura.xaxis.axis_label = "feature " + str(feature1)
figura.yaxis.axis_label = "feature " + str(feature2)

figura.circle(entradas[:, feature1], 
              entradas[:, feature2], 
              color=salidas_a_colores)
show(figura)

Ejercicio 1
===========

Modificar el código que grafica las clases en base a dos dimensiones, para que arme un gráfico por cada posible par de dimensiones (features).

Ejercicio 2
===========

Viendo todos los gráficos del ejercicio 1, ¿alguno de ellos permite saber que los datos son fácilmente clasificables (separables)? ¿Con cuáles dimensiones se ve eso? (responder editando esta misma celda)

---

En clases aprendimos sobre la necesidad de normalizar los datos cuando hay features que varían mucho en rango de valores. Esto es fácilmente realizable utilizando la clase StandardScaler que nos provee Scikit learn.

Como nuestros datos poseen rangos muy diferentes para cada feature, vamos a normalizarlos antes de utilizarlos.


Ejercicio 3
===========

Completar la siguiente celda de código para que la variable entradas_normalizadas contenga los datos de entrada normalizados:

In [ ]:
entradas_normalizadas =   # entradas normalizadas

---

En clases aprendimos sobre los riesgos del sobreentrenamiento (overfitting), y que este problema se combate separando nuestros datos en un set de entrenamiento y otro de test. Scikit learn tiene funcionalidad específica para separar sets de datos de esa manera, la función ```train_test_split```.

Ejercicio 4
===========

Completar la siguiente celda de código para que las variables contengan los sets generados con la función ```train_test_split``` (**IMPORTANTE**: recuerden que los datos que tiene que usar para las entradas son los de la variable entradas_normalizdas):

In [ ]:
entradas_entrenamiento, entradas_test, salidas_entrenamiento, salidas_test =  # llamada a train_test_split

Ahora podemos crear un clasificador de tipo K-vecinos, y entrenarlo con los datos del set de entrenamiento. Como sabemos, k-vecinos requiere que se le defina el parámetro K (la cantidad de vecinos a observar al predecir), y desconocemos un valor adecuado para K. Pero para realizar una prueba inicial, elegimos el valor 1.

In [ ]:
clasificador = KNeighborsClassifier(n_neighbors=1)

Ejercicio 5
========

Con los datos listos y el clasificador ya instaciado, ahora debemos entrenarlo. Completar la siguiente celda para que el clasificador ejecute el entrenamiento (utilizando las entradas y salidas de entrenamiento):

In [ ]:
# código que entrena al clasificador

Y ahora podemos usarlo para predecir salidas a partir de entradas. Vamos a predecir las salidas de las entradas de los dos conjuntos:

In [ ]:
salidas_predichas_entrenamiento = clasificador.predict(entradas_entrenamiento)
salidas_predichas_test = clasificador.predict(entradas_test)

Con las salidas predichas podemos comparar la precisión, accuracy y recall en cada uno de los dos conjuntos, comparando las predicciones con las salidas reales que esperábamos:

In [ ]:
conjuntos_salidas = (
    ('entrenamiento', salidas_entrenamiento, salidas_predichas_entrenamiento),
    ('test', salidas_test, salidas_predichas_test),
)
medidas = (precision_score, accuracy_score, recall_score)

for nombre, salidas, salidas_predichas in conjuntos_salidas:
    print 'Set de', nombre
    for medida in medidas:
        print '   ', medida.func_name, ':', medida(salidas, salidas_predichas)

Ejercicio 6
===========

¿Por qué al medir la performance en el conjunto de entrenamiento, las predicciones fueron perfectas? (1.0 en todas las métricas de ese conjunto) (responder editando esta misma celda)

---

Siendo que el resultado en el conjunto de test (probablemente) se pueda mejorar con un valor de K elegido a consciencia, nos interesaría probar con otros valores de K y evaluar la performance con esos otros valores. Y no solo con algunos valores elegidos al azar, sino que nos interesa evaluar el algoritmo para todos los posibles valores de K desde 1 hasta 150.

Vamos a crear algunas variables donde ir guardando las métricas para cada valor de K:

In [ ]:
precisiones = []
accuracies = []
recalls = []

Ejercicio 7
===========

Escribir código que vaya desde K=1 hasta K=150, y en cada iteración cree un clasificador con el valor correspondiente de K, lo entrene, lo use para predecir las salidas del conjunto de test, evalúe precisión, accuracy y recall en esas predicciones, y guarde las 3 métricas en las variables anteriores, usando K como índice. Ej:

    precisiones[11] = el número que dio la precisión del clasificador con K=12

Estas métricas deben ser evaluadas **solo** para el conjunto de test.

In [ ]:
# código que rellena los valores en precisiones, accuracies y recalls

Con los valores completos, podemos ahora graficar las métricas de performance con respecto a los valores posibles de K:

In [ ]:
ks = range(1, 101)

datos_medidas = (
    (u'Precisión', 'red', precisiones),
    (u'Accuracy', 'green', accuracies),
    (u'Recall', 'blue', recalls),
)

figura = figure()

figura.xaxis.axis_label = 'K'

for nombre, color, valores in datos_medidas:
    figura.line(ks, valores,
                color=color,
                legend=nombre)

show(figura)

Ejercicio 8
===========

Habiendo visualizado la performance para los distintos valores de K, ¿qué valor de K elegirían como el más adecuado para utilizar? ¿Por qué? (responder editando esta misma celda)

Ejercicio 9
===========

El valor de precisión obtenido para ese K, ¿es válido como estimado de la precisión que tendríamos prediciendo futuras muestras que no hayan estado en los datos que utilizamos en esta entrega? ¿Por qué? ¿Cómo estimarían dicha precisión? (responder editando esta misma celda)